In [1]:
from src.chinese_checkers.experience import Experience, ExperienceGenerator, IRewardStrategy, RewardStrategyFactory
from src.chinese_checkers.encoder import EncoderRegistry, EncoderMetadata
from src.chinese_checkers.simulation import SimulationCatalog, SimulationMetadata, GameSimulation

from tqdm import tqdm
from typing import List


# Encoder Module Examples

---

## Example 1: Load simulation and encode them

In [2]:
catalog = SimulationCatalog()
dataset_metadata: List[SimulationMetadata] = catalog.list_datasets()
dataset_metadata

[SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bootstrap-simulation', version='v0.0.1'),
 SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bootstrap-simulation', version='v0.0.1')]

In [3]:
winning_players = ["0","3"] 
player_count = 2
board_size = 4

simulations: List[GameSimulation] = [
    dataset
    for metadata in dataset_metadata
    if metadata.winning_player in winning_players
        and metadata.player_count == player_count
        and metadata.board_size == board_size
    for dataset in catalog.load_simulations_by_metadata(metadata)
][:10]
print(f"Found {len(simulations)} datasets for winning_players: {winning_players} and player_count: {player_count}.")

Found 1704 datasets for winning_players: ['0', '3'] and player_count: 2.


In [4]:
# use the distance to win simulation strategy
reward_strategy: IRewardStrategy = RewardStrategyFactory.create("distance_to_win")
experience_generator = ExperienceGenerator(reward_strategy)

In [5]:
# Create new experiences from the simulations 
experiences: List[Experience] = [
    exper
    for sim in tqdm(simulations, desc="Generating experiences from simulations")
    for exper in experience_generator.generate_experiences_from_simulation(sim)
]

# Print the number of experiences extracted
print(f"Total experiences extracted: {len(experiences)}")

# Display a few sample experiences
for e in experiences[:4]:
    print(e)

Generating experiences from simulations: 100%|██████████| 1704/1704 [00:11<00:00, 143.49it/s]


Total experiences extracted: 10
Experience(state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645E930>, action=Move(-1, 0), Position(3, 2), reward=0.002881844380403458, next_state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645E900>, done=True)
Experience(state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645E900>, action=Move(1, 0), Position(-3, -2), reward=-0.005763688760806916, next_state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645F920>, done=True)
Experience(state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645F920>, action=Move(-2, -2), Position(3, 4), reward=0.008645533141210375, next_state=<src.chinese_checkers.game.ChineseCheckersGame.ChineseCheckersGame object at 0x000002565645F980>, done=True)
Experience(state=<src.chinese_checkers.game.ChineseCheckersGame.C

In [7]:

# create an encoder
encoder_metadata = EncoderMetadata(
    encoder_name="experience-encoder-v0.0.1",
    board_size=board_size
)
encoder = EncoderRegistry.get_encoder(encoder_metadata)

print(encoder.encode(experiences[5]))

RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [ ]:
from tqdm import tqdm
import numpy as np

# Load catalog and metadata
catalog = SimulationCatalog("D:/chinese_checkers_games")
metadata_list = catalog.list_available_metadata()

# Check if metadata is available
if not metadata_list:
    print("No metadata found in the catalog.")
else:
    metadata = metadata_list[0]  # Select the first metadata entry or choose as needed

    # Load encoded experiences from the catalog
    loaded_experiences = catalog.load_experiences_by_metadata(metadata)

    print(f"Loaded {len(loaded_experiences)} experiences from the catalog.")

    # Process loaded experiences if needed
    for exp in tqdm(loaded_experiences, desc="Processing loaded experiences", unit="experience"):
        state, action, reward, next_state, done = exp
        # Example: print out the experience components
        print("State:", state)
        print("Action:", action)
        print("Reward:", reward)
        print("Next State:", next_state)
        print("Done:", done)

    # Optionally, you can decode or use the loaded experiences with your encoder or model.
